<a href="https://colab.research.google.com/github/catPotat/EV-classifier/blob/main/EV_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About

#### Keras utilization of MobileNetV2 to recognize Eeveelution types
(90% val_accuracy, feel free to add any comment Ctrl+Alt+M)

 
By: *Reeeon*
 
Thanks to:
* *Sap1231 for eight thousands pics*
* *Eevee Hub#eeveelution-pictures for three thousands pics*

All them pics labeled: https://drive.google.com/drive/folders/1pCc9NoDyo3GAcUqXEI6k1Z5lOm1hsDdw

Trained weights: https://drive.google.com/drive/folders/1ZG8oJHYzCCzg6e848MopcKjqQBHvfqvQ

 
<img src="https://drive.google.com/uc?id=1KTToAwFHae2gCGcjZb7_CDn12xq003B2" alt="cute umbreon picture" height>

# 🏁 Initialize

In [ ]:
!python -V
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir -p datasets/{fragments,Labeled}/
!unzip "drive/My Drive/datasets/Eeveelutions/sap1231-251020.zip" -d datasets/fragments/
!mv datasets/fragments/sap1231-251020/Labeled/* datasets/Labeled/
!unzip "drive/My Drive/datasets/Eeveelutions/evhub#evlution-pics.zip" -d datasets/fragments/
!rsync -abv datasets/fragments/evhub#evlution-pics/Labeled/* datasets/Labeled/

output.clear()
print("Datasets loaded!")


In [ ]:
from google.colab import output
import tensorflow as tf
from tensorflow import keras

print(f'Tensorflow version: {tf.__version__}')
# tf.python.client.device_lib.list_local_devices()
print("# GPUs Available: ",
    len(tf.config.experimental.list_physical_devices("GPU"))
)


# 📈 Evaluation

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs/

👇 *Test and tune then re-run from here (Ctrl+F10)*

# 🔢 Some hyperparams or somethin

In [ ]:
DATA_PATH = "datasets/Labeled/SoloEevee/"
MODEL_PATH = "drive/My Drive/Colab Notebooks/EV-classifier/EV_trained/"
VAL_SPLIT = 0.10
IMG_HEIGHT = 299
IMG_WIDTH = 299
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.00001
SEED = 6110

!ls $DATA_PATH

In [ ]:
# https://www.tensorflow.org/tutorials/images/classification

import time
from tensorflow.keras.callbacks import TensorBoard


timestamp = int(time.time())
tensorboard = TensorBoard(log_dir=f'logs/eevee/{timestamp}')
print(f'Timestamp: {timestamp}')

# 🗂️ Prepare thee dataset


In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory


train_ds = image_dataset_from_directory(
    DATA_PATH,
    # label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    interpolation="bilinear",
    follow_links=True
)
val_ds = image_dataset_from_directory(
    DATA_PATH,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    follow_links=True
)
print(train_ds.class_names)


AUTOTUNE = tf.data.experimental.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


# 🧠 Define the model

In [ ]:
from tensorflow.keras import layers


data_augmentation = keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal", seed=SEED,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.experimental.preprocessing.RandomContrast(0.2, seed=SEED),
    # layers.experimental.preprocessing.RandomRotation(0.1, seed=SEED),
    # layers.experimental.preprocessing.RandomTranslation(0.1, 0.1, seed=SEED),
    # layers.experimental.preprocessing.RandomZoom(0.1, seed=SEED),
    layers.experimental.preprocessing.RandomWidth(0.1, seed=SEED),
])

# https://keras.io/api/applications/ "Fine-tune InceptionV3 on a new set of classes"
# https://keras.io/api/applications/mobilenet/
model = keras.Sequential([
    data_augmentation,
    layers.experimental.preprocessing.Rescaling(
        scale=1./127.5, offset=-1, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    keras.applications.mobilenet_v2.MobileNetV2(
        alpha=0.5,
        weights="imagenet",
        include_top=False,
        pooling="avg",
    ),
    layers.Dropout(0.1, seed=SEED),
    # layers.GlobalAveragePooling2D(),
    # layers.Dense(128, activation='relu'),
    layers.Dense(9, activation='softmax')
])

model.summary()


# 🧮 Pick your optimizer, loss fn, etc

In [ ]:
# https://keras.io/api/models/model_training_apis/

model.compile(
    # optimizer=keras.optimizers.SGD(lr=0.001, momentum=0.0),
    # loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(lr=LEARNING_RATE),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

print(f'Default GPU Device: {tf.test.gpu_device_name()}')

# **🚧 Training**

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[tensorboard],
)

model.save(f'EV-classify-{timestamp}.keras') # backup
print("Model exported!")
!mkdir -p "$MODEL_PATH/$timestamp"
model.save(f'{MODEL_PATH}/{timestamp}/EV-classify-{timestamp}.keras')

In [ ]:
!pip install tensorflowjs
output.clear()

import tensorflowjs as tfjs

tfjs.converters.save_keras_model(model, f'{MODEL_PATH}/{timestamp}/')
print("JSON model saved!")